In [1]:
import warnings
import gc
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.regression.quantile_regression import QuantReg
import matplotlib.pyplot as plt


In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
enem19 = pd.read_parquet('/content/drive/MyDrive/Gabinete/Relatório/Dados/enem_cleared_2019.parquet')

In [ ]:
enem20 = pd.read_parquet('/content/drive/MyDrive/Gabinete/Relatório/Dados/enem_cleared_2020.parquet')

In [ ]:
enem21 = pd.read_parquet('/content/drive/MyDrive/Gabinete/Relatório/Dados/enem_cleared_2021.parquet')

## Funções

In [ ]:
def fit_model(model, q):
    res = model.fit(
        q=q, 
        vcov='robust',
        kernel='epa',
        bandwidth='hsheather', 
        max_iter=1000,
        p_tol=1e-06
    )

    return res.summary()

In [ ]:
def create_model(df, cols):
  mod = smf.quantreg(cols, data=df, missing='drop')
  return mod

### Remoção de caracteres proibidos dos nomes das colunas

---



In [ ]:
enem19.head()

,NU_ANO,LOG_NT_MT,NU_NOTA_LC,NU_NOTA_MT,Q005,NT_MEDIA,LOG_NT_MEDIA,TP_FAIXA_ETARIA_1,TP_FAIXA_ETARIA_2,TP_FAIXA_ETARIA_3,TP_FAIXA_ETARIA_4,TP_FAIXA_ETARIA_5,TP_FAIXA_ETARIA_6,TP_FAIXA_ETARIA_23-25,TP_SEXO_F,TP_ESTADO_CIVIL_0,TP_ESTADO_CIVIL_1,TP_ESTADO_CIVIL_2,TP_ESTADO_CIVIL_3,TP_ESTADO_CIVIL_4,TP_COR_RACA_0,TP_COR_RACA_1,TP_COR_RACA_2,TP_COR_RACA_3,TP_COR_RACA_4,TP_COR_RACA_5,TP_ESCOLA_1,TP_ESCOLA_2,TP_ESCOLA_3,TP_LOCALIZACAO_ESC_1,TP_LOCALIZACAO_ESC_2,Q001_A,Q001_B,Q001_C,Q001_D,Q001_E,Q001_F,Q001_G,Q001_H,Q002_A,Q002_B,Q002_C,Q002_D,Q002_E,Q002_F,Q002_G,Q002_H,Q003_A,Q003_B,Q003_C,Q003_D,Q003_E,Q003_F,Q004_A,Q004_B,Q004_C,Q004_D,Q004_E,Q004_F,Q006_A,Q006_B,Q006_C,Q006_D,Q006_E,Q006_F,Q006_G,Q006_H,Q006_I,Q006_J,Q006_K,Q006_L,Q006_M,Q006_N,Q006_O,Q006_P,Q006_Q,Q007_A,Q007_B,Q007_C,Q007_D,Q008_A,Q008_B,Q008_C,Q008_D,Q008_E,Q009_A,Q009_B,Q009_C,Q009_D,Q009_E,Q010_A,Q010_B,Q010_C,Q010_D,Q010_E,Q012_A,Q012_B,Q012_C,Q012_D,Q012_E,Q019_A,Q019_B,Q019_C,Q019_D,Q019_E,Q022_A,Q022_B,Q022_C,Q022_D,Q022_E,Q024_A,Q024_B,Q024_C,Q024_D,Q024_E,Q025_1
9,2019.0,6.175243,502.5,480.7,5.0,495.04,6.204639,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
11,2019.0,6.129050,417.6,459.0,4.0,449.30,6.107691,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
13,2019.0,5.988961,413.0,399.0,4.0,390.50,5.967428,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
26,2019.0,6.570463,636.3,713.7,2.0,722.58,6.582828,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
27,2019.0,6.304814,489.9,547.2,3.0,507.22,6.228945,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
rename_cols = {col: col.replace('-','a') for col in enem19.columns if '-' in col}

In [ ]:
enem19.rename(columns=rename_cols, inplace=True)

### Selecionando alunos de escola pública, de áreas urbanas e rurais separadamente

In [ ]:
pub19 = enem19[enem19.TP_ESCOLA_2==1].drop(columns=['TP_ESCOLA_1','TP_ESCOLA_2','TP_ESCOLA_3'])
urb_pub19 = pub19[pub19['TP_LOCALIZACAO_ESC_1']==1].drop(columns=['TP_LOCALIZACAO_ESC_1','TP_LOCALIZACAO_ESC_2'])

In [ ]:
# Isolando presença de internet (sem contar para dispositivo)
urb_pub19 = urb_pub19.drop(columns=[col for col in urb_pub19.columns if col.startswith('Q024') or col.startswith('Q022')])

## Treinando modelo

In [ ]:
def run_training(df, y_col, file_name):
  independent_vars = ['NU_ANO', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NT_MEDIA', 'LOG_NT_MEDIA', 'LOG_NT_MT']
  cross = f"{y_col} ~ {' + '.join(col for col in df.columns if col not in independent_vars)}"
  mod = create_model(df, cross)

  # Fit no modelo
  quantiles = np.arange(0.05, 0.96, 0.1)

  models = {x: fit_model(mod, x) for x in quantiles}

  # Persistindo resultados
  dfs = []
  metadatas = []  

  for k, v in models.items():
    # Note that tables is a list. The table at index 1 is the "core" table. Additionally, read_html puts dfs in a list, so we want index 0
    results_as_html = v.tables[1].as_html()
    metadata = v.tables[0].as_html()
    metadata = pd.read_html(metadata, header=0, index_col=0)[0]
    df = pd.read_html(results_as_html, header=0, index_col=0)[0]
    df['quantile'] = k
    metadata['quantile'] = k
    dfs.append(df)
    metadatas.append(metadata)
  
  all = pd.concat(dfs)
  meta = pd.concat(metadatas, axis=1)

  # Salvando localmente resultados para cada quantil
  all.to_csv(f'/content/drive/MyDrive/Gabinete/Relatório/Results/resultQReg{file_name}')
  meta.to_csv(f'/content/drive/MyDrive/Gabinete/Relatório/Results/metaQReg{file_name}')

  return all, meta

In [ ]:
r19, meta19 = run_training(df=pub19, y_col='LOG_NT_MT', file_name='19-Publicas-Urbanas.csv')

ValueError: ignored

In [ ]:
independent_vars = ['NU_ANO', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NT_MEDIA', 'LOG_NT_MEDIA', 'LOG_NT_MT']

In [ ]:
ntMatXall = f"LOG_NT_MT ~ {' + '.join(col for col in urb_pub19.columns if col not in independent_vars)}"

In [ ]:
mod = create_model(urb_pub19, ntMatXall)

In [ ]:
res = mod.fit(
    q=0.5, 
    vcov='robust',
    kernel='gau',
    bandwidth='hsheather', 
    max_iter=1000,
    p_tol=1e-06
)

ValueError: ignored

In [ ]:
quantiles = np.arange(0.05, 0.96, 0.1)

models = {x: fit_model(mod, x) for x in quantiles}

ValueError: ignored

In [ ]:
dfs = []
metadatas = []

In [ ]:
for k, v in models.items():
  # Note that tables is a list. The table at index 1 is the "core" table. Additionally, read_html puts dfs in a list, so we want index 0
  results_as_html = v.tables[1].as_html()
  metadata = v.tables[0].as_html()
  metadata = pd.read_html(metadata, header=0, index_col=0)[0]
  df = pd.read_html(results_as_html, header=0, index_col=0)[0]
  df['quantile'] = k
  metadata['quantile'] = k
  dfs.append(df)
  metadatas.append(metadata)

In [ ]:
all = pd.concat(dfs)
meta = pd.concat(metadatas, axis=1)

In [ ]:
all.to_csv('/content/drive/MyDrive/Gabinete/Relatório/Results/resultadoQReg19-Publicas-Urbanas.csv')
meta.to_csv('/content/drive/MyDrive/Gabinete/Relatório/Results/metaQReg19-Publicas-Urbanas.csv')
